# Overview
- This notebook intends to walk through the raw data saved in `data/pull`, outlining potential issues and pitfalls. 
- After running `doit`, your local `data/pull` directory should have four files:
    1. `CRSP_fund_combined.parquet`: CRSP monthly mutual funds data.
    2. `s12.parquet`: S12 quarterly mutual fund holdings data. 
    3. `mflink1.parquet`: To link `crsp_fundno` with `wficn`. 
    4. `mflink2.parquet`: To link S12's `fundno` with `wficn`. 

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import wrds

from load_CRSP_fund import load_CRSP_combined_file
from load_s12 import load_s12_file
from load_mflink import load_mflink1, load_mflink2


import config
WRDS_USERNAME = config.WRDS_USERNAME

df_crsp = load_CRSP_combined_file()
df_s12 = load_s12_file()
df_mflink1 = load_mflink1()
df_mflink2 = load_mflink2()

# CRSP Mutual Fund Data

## Tables and Filters
- The main data is pulled from `crsp.monthly_tna_ret_nav`. (https://wrds-www.wharton.upenn.edu/data-dictionary/crsp_q_mutualfunds/monthly_tna_ret_nav/)
- The paper specifies that it utilized only "US domestic equity" funds, so we need to identify this information. 
- Through various trials and errors, I found out that the best way to achieve this filter is through the `crsp.fund_style` table's `crsp_obj_cd` column. (https://wrds-www.wharton.upenn.edu/data-dictionary/crsp_q_mutualfunds/fund_style/)
- I left join these two tables above. 
- Based on the CRSP manual, I require the first two characters of this code to be "ED", representing "Equity" and "Domestic". 

## Obtaining `wficn`
- I discovered that **each mutual fund can have multiple `crsp_fund_no`**, representing different _share classes_. 
- It is of critical important for us to obtain the `wficn`, which is a fund-level identifier, and then aggregate the results. 
- The author describes the algorithm to perform this aggregation in **footnote 4**. 

## Multiple `wficn` for each `crsp_fundno`
- As explained above, it's very common for one `wficn` to match with multiple `crsp_fundno`, because the latter represents a specific share class of a fund. 
- However, I discovered rare occurrances where one `crsp_fundno` matches with multiple `wficn`. This is unexpected, since each `wficn` should conceptually represents one "institution" or "fund". 
- I could not figure out the underlying reasons, but suspect that it could have something to do with delisting / merging of funds. For instance, one fund could be fully aqcuired by another fund, and thus assumed two fund identifiers. 

In [2]:
df_mflink1.groupby("crsp_fundno").size().value_counts()

1    48981
2      273
3       41
4        4
Name: count, dtype: int64

- It is also important to point out that certain `crsp_fundno` cannot be matched with any `wficn`. 
- Based on the descriptions in the paper, I decide to drop these samples. 
- Next, let us merge CRSP data and `mflink1` to obtain the appropriate `wficn`. 

In [3]:
print(f"Before merging, df_crsp has {df_crsp.shape[0]} rows")
df_crsp = df_crsp.merge(df_mflink1, how="inner", on="crsp_fundno").reset_index(
    drop=True
)
print(f"After merging, df_crsp has {df_crsp.shape[0]} rows")

Before merging, df_crsp has 3555788 rows


After merging, df_crsp has 3270889 rows


In [4]:
df_crsp = df_crsp.sort_values(["caldt", "wficn"])
df_crsp['year'] = df_crsp['caldt'].dt.year.astype('int')
df_crsp['month'] = df_crsp['caldt'].dt.month.astype('int')
df_crsp = df_crsp[df_crsp['wficn'].notnull()]
df_crsp['wficn'] = df_crsp['wficn'].astype('int')
df_crsp['mret'] = df_crsp['mret'].fillna(0)
df_crsp.head()

,caldt,crsp_fundno,mtna,mret,mnav,lipper_asset_cd,lipper_class_name,crsp_obj_cd,index_fund_flag,wficn,year,month
2035300,1980-01-31,32668.0,NaN,0.000000,NaN,None,None,EDYG,None,100007,1980,1
10042,1980-01-31,967.0,NaN,0.121324,12.20,None,None,EDYG,None,100010,1980,1
344023,1980-01-31,7353.0,NaN,0.056240,24.38,None,None,EDYG,None,100019,1980,1
15191,1980-01-31,1445.0,NaN,0.016978,5.99,None,None,EDYG,None,100046,1980,1
3248,1980-01-31,297.0,NaN,0.079268,17.70,None,None,EDYG,None,100056,1980,1


## Computing Yearly Returns
- To replicate Table 1, we need to compute yearly returns. 
- To do that, we first need to compute each fund's monthly returns. 
- **Intention**: We would like to follow footnote 4's approach of using `mtna` as weight. 
- **Issue**: Not all `mtna` are available, most likely because the mutual funds did not report this number. This is especially severe for 1990 and earlier. 
- **Solution**: The paper does not specify the method of resolving this issue. We could pull in TNA values elsewhere. Here, I will simply use **simple average** instead. This is reasonable, because it's most likely that different share classes of the same mutual fund should have very close returns. 

In [5]:
df_crsp['mtna'].isnull().mean()

0.020550376365569115

In [6]:
df_ret = (
    df_crsp.groupby(
        [
            "wficn",
            "year",
            "month",
        ]
    )["mret"]
    .mean()
    .reset_index()
)
df_ret["mult"] = 1 + df_ret["mret"]
df_ret["cumret"] = (
    df_ret.sort_values(["year", "month"]).groupby(["wficn", "year"])["mult"].cumprod()
)

# only care about yearly return
df_ret = df_ret.query("month==12")
df_ret['yret'] = df_ret['cumret'] - 1


In [7]:
df_ret.head()

,wficn,year,month,mret,mult,cumret,yret
0,100001,1990,12,0.027962,1.027962,1.027962,0.027962
12,100001,1991,12,0.115646,1.115646,1.301927,0.301927
24,100001,1992,12,0.000407,1.000407,1.057012,0.057012
36,100001,1993,12,0.007705,1.007705,1.060564,0.060564
48,100001,1994,12,0.017971,1.017971,0.986237,-0.013763


## Year-end TNA
- Table 1 reports TNA, but it does not specify whether it is average of max or year-end TNA. 
- For this project, I will use year-end TNA only. 
- We then merge the TNA and yearly return information. 

In [27]:
df_tna = df_crsp.query("month==12").groupby(["wficn", "year"])["mtna"].sum().reset_index().rename(columns={"mtna": "crsp_tna"})
df_crsp_clean = pd.merge(df_tna, df_ret)[['wficn', 'year', 'crsp_tna', 'yret']]


In [9]:
df_crsp_clean.head()

,wficn,year,crsp_tna,yret
0,100001,1990,169.570,0.027962
1,100001,1991,330.031,0.301927
2,100001,1992,596.275,0.057012
3,100001,1993,857.674,0.060564
4,100001,1994,876.194,-0.013763


In [36]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
float_format_func = lambda x: '{:.2f}'.format(x)
df_crsp_clean = df_crsp_clean.rename(columns={'crsp_tna': 'crsp_{tna}'})
latexTS_crsp_clean = df_crsp_clean.head(5).to_latex(float_format = float_format_func)

path_to_save = f'../reports/table_crsp_clean.tex'

with open(path_to_save, 'w') as f: 
    f.write(latexTS_crsp_clean)


# S12 Data
- The S12 database link: https://wrds-www.wharton.upenn.edu/data-dictionary/tr_mutualfunds/s12/

## Missing TNA Values
- The author specified in the paper that "we require that the TNAs reported in the Thomson Reuters database and in the CRSP database do not di￿er by more than a factor of two."
- Looking at s12 table, it's clear that `assets` represents the TNA values. 
- **Issues**: The s12 table has a lot of missing assets fields for 2010 and 2013, especially 2011 and 2012. See below for a demonstration.  
- **Solution**: I cannot think of any obvious solution. One possibility might be to assume the TNA is merely the sum of all holdings' values provided by the s12 table.

In [10]:
query = """
SELECT
  EXTRACT(YEAR FROM fdate) AS year,
  COUNT(*) AS total_rows,
  COUNT(CASE WHEN assets IS NULL THEN 1 END) AS missing_assets,
  (COUNT(CASE WHEN assets IS NULL THEN 1 END) * 100.0 / COUNT(*)) AS missing_percentage
FROM
  tfn.s12
WHERE
  fdate >= '2007-01-01' and fdate <= '2016-12-31'
GROUP BY
  EXTRACT(YEAR FROM fdate)
ORDER BY
  year;
"""

db = wrds.Connection(wrds_username=WRDS_USERNAME)
temp = db.raw_sql(query)
db.close()


Loading library list...
Done


In [11]:
temp

,year,total_rows,missing_assets,missing_percentage
0,2007.0,4365627,688,0.015759
1,2008.0,4812433,2405,0.049975
2,2009.0,4792996,926,0.019320
3,2010.0,4343944,822231,18.928214
4,2011.0,4097533,3743918,91.370051
5,2012.0,4135072,3939564,95.271957
6,2013.0,4139295,2457418,59.368032
7,2014.0,4405473,145486,3.302392
8,2015.0,4900764,3059,0.062419
9,2016.0,5249452,8173,0.155692


## Merging `s12` and `mflink2`
- Unlike `mflink1`, the mapping for s12 to wficn has date information. 
- If I simply use the tuple of (fdate, fundno) to merge, there will be a lot of missing matches. 
- To circumvent this issue, I decided to **obtain the last valid record of wficn for each (year, fundno)**. 
- During our meeting with Jeremy, he suggested we use `merge_asof` instead, which makes more sense. 
- We observe a huge reduction in sample size after the merge, probably because s12 contain a lot of **non domestic funds** which are not covered WRDS's MFLINK

In [12]:
print(f"Before merge: {df_s12.shape[0]}")
df_s12["year"] = df_s12["fdate"].dt.year.astype("int")
df_s12 = df_s12.sort_values(["fdate", "fundno"])
df_mflink2 = df_mflink2.sort_values(["fdate", "fundno"])
df_s12['fundno'] = df_s12['fundno'].astype('int')
df_mflink2['fundno'] = df_mflink2['fundno'].astype('int')
df_s12 = pd.merge_asof(df_s12, df_mflink2[['fdate', 'fundno', 'wficn']], by='fundno', on='fdate', direction='nearest')
df_s12 = df_s12[df_s12['wficn'].notnull()]
print(f"After merge: {df_s12.shape[0]}")

Before merge: 6394435
After merge: 1918915


## Domestic Equity?
- S12 data has a "country" to identify countries of the stocks, and "stkcdesc" to identify classes of the stocks. 
- However, these data are not missing before 2000.
- Since we've already filtered on domestic equity funds in `df_crsp_clean`, I decided to just **group together** all holdings and assume they are all US equities. 

In [13]:
# temporarily fillna with 0 to avoid missing records
df_s12['assets'] = df_s12['assets'].fillna(0)
df_eq = df_s12.groupby(['year', 'fdate', 'wficn', 'assets', ])['useq_tna_k'].sum().reset_index()

- As explained before, I am not aware of any clear solution for replacing `assets`, and I will simply keep it as NaN for now. 

In [14]:
df_eq['assets'] = np.where(df_eq['assets'] == 0, np.nan, df_eq['assets'])

- `s12` is updated quarterly, but the paper does not specify how to aggregate on a yearly basis. 
- For simplicity, I will simply get the last record for each year for now. 

In [15]:
df_eq.head()

,year,fdate,wficn,assets,useq_tna_k
0,1980,1980-03-31,100007.0,235.0,2997.29000
1,1980,1980-03-31,100010.0,5967.0,49518.54094
2,1980,1980-03-31,100019.0,6825.0,44089.73500
3,1980,1980-03-31,100046.0,3313.0,24476.75100
4,1980,1980-03-31,100050.0,3319.0,2940.38700


In [16]:
df_eq = df_eq.groupby(['wficn', 'year'])[['assets', 'useq_tna_k']].last().reset_index()

# Merging CRSP and S12 Data
- It is finally time to merge. 

In [17]:
df_eq.head()

,wficn,year,assets,useq_tna_k
0,100001.0,1990,16957.0,161803.10000
1,100001.0,1991,33003.0,314952.40000
2,100001.0,1992,59627.0,578201.50000
3,100001.0,1993,84286.0,821482.00000
4,100001.0,1994,92961.0,896403.42887


In [18]:
df_crsp_clean.head()    

,wficn,year,crsp_tna,yret
0,100001,1990,169.570,0.027962
1,100001,1991,330.031,0.301927
2,100001,1992,596.275,0.057012
3,100001,1993,857.674,0.060564
4,100001,1994,876.194,-0.013763


In [19]:
df_combo = pd.merge(df_crsp_clean, df_eq, on=["wficn", "year"], how="inner").sort_values("year")
df_combo.head()

,wficn,year,crsp_tna,yret,assets,useq_tna_k
1460,100207,1980,26.689,0.449720,2622.0,20617.23075
1498,100209,1980,25.852,0.285324,2556.0,23088.36700
46575,200257,1980,665.568,0.258873,55751.0,486735.91956
23458,103179,1980,125.000,0.785458,12550.0,115753.72500
20579,102774,1980,68.099,0.298462,6810.0,54214.32000


# Applying Filters To Identify Universe
- Let us first see the universe before applying any filters.
- It is interesting to observe that without applying any filters, this number of funds data match closely for certain years, but mismatch greatly for others. 
- For 1980 and 1993, for instance, the numbers are identical. 

In [20]:
df_combo.groupby('year').size().reset_index().rename(columns={0: 'count'})

,year,count
0,1980,196
1,1981,189
2,1982,193
3,1983,221
4,1984,234
5,1985,252
6,1986,295
7,1987,338
8,1988,368
9,1989,420


- Next, I will apply some filters as specified in the paper. 

In [21]:
# TNA above 1 million at year end
df_combo = df_combo.query("crsp_tna > 1")
df_combo.groupby("year").size().reset_index().rename(columns={0: "count"})

,year,count
0,1980,193
1,1981,186
2,1982,191
3,1983,220
4,1984,233
5,1985,251
6,1986,293
7,1987,336
8,1988,366
9,1989,417


- Since we have a lot of missing "assets" for certain years, for simplicity, I assume the TNA ratio would be 1 in that case. 

In [22]:
df_combo["tna_ratio"] = np.where(
    df_combo["assets"].isnull(),
    1,
    df_combo["crsp_tna"] * 1e6 / df_combo["assets"] / 1e4,
)
df_combo = df_combo.query("tna_ratio > 0.5 and tna_ratio < 2")
df_combo.groupby("year").size().reset_index().rename(columns={0: "count"})

,year,count
0,1980,192
1,1981,186
2,1982,190
3,1983,218
4,1984,233
5,1985,250
6,1986,287
7,1987,329
8,1988,359
9,1989,405


- Finally we compute the equity ratio. 
- The paper doesn't specify which TNA to use. 
- I will use both and require at least one of them to fall between 0.8 and 1.05.

In [23]:
df_combo["eq_ratio_1"] = df_combo["useq_tna_k"] * 1e3 / (df_combo["crsp_tna"] * 1e6)
df_combo["eq_ratio_2"] = np.where(
    df_combo["assets"].isnull(),
    1,
    df_combo["useq_tna_k"] * 1e3 / (df_combo["assets"] * 1e4),
)
df_combo.head()

,wficn,year,crsp_tna,yret,assets,useq_tna_k,tna_ratio,eq_ratio_1,eq_ratio_2
1460,100207,1980,26.689,0.449720,2622.0,20617.23075,1.017887,0.772499,0.786317
1498,100209,1980,25.852,0.285324,2556.0,23088.36700,1.011424,0.893098,0.903301
46575,200257,1980,665.568,0.258873,55751.0,486735.91956,1.193823,0.731309,0.873053
23458,103179,1980,125.000,0.785458,12550.0,115753.72500,0.996016,0.926030,0.922340
20579,102774,1980,68.099,0.298462,6810.0,54214.32000,0.999985,0.796110,0.796099


In [24]:
df_combo = df_combo[
    (df_combo["eq_ratio_1"].between(0.8, 1.05))
    | (df_combo["eq_ratio_2"].between(0.8, 1.05))
]
df_combo.groupby("year").size().reset_index().rename(columns={0: "count"})

,year,count
0,1980,142
1,1981,113
2,1982,125
3,1983,144
4,1984,148
5,1985,175
6,1986,188
7,1987,203
8,1988,244
9,1989,262


- The above filter appears to remove too many samples for most years. 

## Returns and TNA
- It's not easy to match these numbers, especially the return numbers. 

In [25]:
df_combo.groupby("year")[["crsp_tna", "yret"]].agg(["mean", "median"]).reset_index().round(2)

year crsp_tna          yret       
             mean  median  mean median
0   1980   135.38   53.50  0.36   0.36
1   1981   154.00   55.03 -0.04  -0.04
2   1982   169.27   67.98  0.24   0.25
3   1983   228.68   84.00  0.18   0.18
4   1984   225.05   79.50 -0.04  -0.04
5   1985   201.82   91.90  0.27   0.27
6   1986   228.07   83.51  0.12   0.13
7   1987   222.74   66.28  0.00   0.01
8   1988   223.77   64.01  0.14   0.14
9   1989   259.74   71.32  0.25   0.26
10  1990   362.16  106.15 -0.04  -0.03
11  1991   416.54  114.86  0.34   0.31
12  1992   361.51  111.59  0.07   0.07
13  1993   429.68  113.84  0.11   0.10
14  1994   417.67  104.85 -0.01  -0.01
15  1995   661.12  146.66  0.29   0.31
16  1996   882.71  170.90  0.18   0.19
17  1997  1063.56  183.65  0.21   0.24
18  1998  1223.19  171.00  0.13   0.13
19  1999  1390.16  179.75  0.26   0.18
20  2000  1317.95  204.40  0.01  -0.02
21  2001  1034.07  156.35 -0.10  -0.11
22  2002   723.14  124.40 -0.21  -0.22
23  2003  1009.79  168.10  0.34   0.31
24  2004  1137.75  200.10  0.13   0.12
25  2005  1266.91  228.30  0.08   0.07
26  2006  1478.86  257.70  0.15   0.13
27  2007  1248.73  210.30  0.06   0.05
28  2008   782.02  124.90 -0.38  -0.38
29  2009  1120.28  190.35  0.32   0.30
30  2010  1469.46  256.40  0.19   0.17
31  2011  1363.40  217.70 -0.03  -0.02
32  2012  1567.57  243.70  0.14   0.15
33  2013  1998.51  344.90  0.32   0.33
34  2014  2239.81  349.05  0.08   0.08
35  2015  2097.29  290.05 -0.02  -0.02
36  2016  2341.01  287.25  0.12   0.11
37  2017  2293.50  283.70  0.18   0.18
38  2018  2189.51  247.45 -0.08  -0.08
39  2019  2691.27  299.50  0.26   0.26
40  2020  2871.09  344.10  0.17   0.14
41  2021  3439.36  399.20  0.21   0.22
42  2022  2920.07  341.25 -0.17  -0.17
43  2023  3727.34  428.20  0.19   0.17